Configuraçao do Ambiente (ferramentas,base de dados,testes)

In [ ]:
#Ferramentas utilizadas
import pandas as pd #leitura
import matplotlib.pyplot as plt #graficos
import ast #array em formato de string para formato de array normal
from geopy.geocoders import Nominatim #pegar coordenadas
import folium
from unidecode import unidecode
from IPython.display import display



In [ ]:
#Bases de dados importadas
dados_ratings = pd.read_csv('desafio_ratings.csv',sep=';')
dados_details = pd.read_csv('desafio_details.csv',sep=';')

In [ ]:
#Teste ratings
dados_ratings.head()

In [ ]:
#Teste details
dados_details.head()

Análise 1 - Cidades em ordem crescente de número de listings (anúncio ou oferta de uma propriedade)

In [ ]:
#Para isso vamos separar os valores de interesseas duplicações de um mesmo imovel (variados quartos possuem um unico anúncio) 
#e contar quantas vezes uma cidade aparece,o que resulta no número de imoveis por cidade
dados_listings = dados_details.loc[:,('hotel_id','city_name')].drop_duplicates()
dados_listings_ordenados = dados_listings['city_name'].value_counts(ascending=True).reset_index()
dados_listings_ordenados

Análise 2- Cidades em ordem decrescente de metros quadrados

In [ ]:
#Para isso separar os dados em interesse e depois vamos agrupa-los em cidades e somar todos os m2 por meio da funcao 'sum()'
dados_m2 = dados_details.loc[:,('city_name','room_surface_in_m2')]
dados_m2_soma = dados_m2.groupby('city_name').sum().sort_values(by='room_surface_in_m2',ascending=False).reset_index()
dados_m2_soma

Análise 3 - Quais cidades têm mais avaliações?

In [ ]:
#Para isso vamos usar 2 dados em tabelas distintas já que eles estão separados em 2 bancos de dados diferentes
#Vamos usar da desafio_ratings = hotel_id,number_of_ratings e da desafio_details = hotel_id,city_name
#e concilia-lás para termos um dataFrame que junte as cidades aos números de avaliação

dados_hotel_cidade = dados_details.loc[:,('hotel_id','city_name')].drop_duplicates()
dados_hotel_cidade

In [ ]:
dados_hotel_ratings = dados_ratings.loc[:,('hotel_id','number_of_ratings')]
dados_hotel_ratings

In [ ]:
#Agora juntando as 2 com a função 'pd.merge()' com base no hotel_id e após isso agrupar por cidade e somar tudo pela função 'sum()'
dados_hotel_cidade_ratings = pd.merge(dados_hotel_cidade,dados_hotel_ratings,on='hotel_id')
dados_num_avaliacao = dados_hotel_cidade_ratings.loc[:,('city_name','number_of_ratings')].groupby('city_name').sum().sort_values(by='number_of_ratings',ascending=False)
dados_num_avaliacao

Análise 4 - Quais cidades têm a maior média de avaliações? E a menor média?

In [ ]:
#Utiliza os dados encontrados anteriomente 'dados_hotel_cidade_ratings',agrupa-os novamente por cidade
#e em vez de somar tudo,calcula-se a media por meio da funcao 'mean()'
dados_media_avaliacao = dados_hotel_cidade_ratings.loc[:,('city_name','number_of_ratings')].groupby('city_name').mean().sort_values(by='number_of_ratings',ascending=False)
dados_media_avaliacao

Análise 5 - Existem correlações entre as características de um anúncio e a sua localização?

In [ ]:
#Para essa vou verificar essa correlaçao vou analisar uma caracteristica que define demais a experiencia do cliente : tipo de locação

In [ ]:
dados_tipo = dados_details.loc[:,('hotel_id','city_name','accommodation_type')].drop_duplicates()
dados_por_tipo = dados_tipo.loc[:,('city_name','accommodation_type')].value_counts().reset_index()
dados_por_tipo

In [ ]:
# Supondo que seus dados estejam em um DataFrame chamado df
# Substitua df pelo nome real do seu DataFrame

# Crie um DataFrame com as colunas 'city_name', 'accommodation_type' e 'count'
dados_por_tipo = dados_por_tipo[['city_name', 'accommodation_type', 'count']].drop_duplicates()

# Crie uma tabela dinâmica para obter a contagem total por tipo de acomodação e cidade
accommodation_counts = pd.pivot_table(dados_por_tipo, values='count', index='city_name', columns='accommodation_type', fill_value=0)

# Adicione uma coluna indicando se há mais apartamentos do que a soma das outras locações
accommodation_counts['more_apartments_than_others'] = accommodation_counts['apartment'] > accommodation_counts.drop('apartment', axis=1).sum(axis=1)

# Exiba o resultado
accommodation_counts.reset_index(inplace=True)


In [ ]:
#configurar o geolocalizador sem verificação SSL
geolocator = Nominatim(user_agent="my_geocoder", ssl_context=None)

#função para mapear a nota para uma cor
def map_nota_to_color(tipo):
    # Usei um gradiente de azul para verde para vermelho
    if not tipo:
        return 'blue'
    else:
        return 'red'

#função para manipular o nome da cidade
def format_city_name(city):

    #remover acentos e converter para minúsculas e adicionando brasil para diminuir erros relativos a localização
    formatted_city = unidecode(city).lower()+ '-brasil'

    #cidades que não condiziam com a localização encontrada e precisaram de uma ajuda na pesquisa
    if formatted_city == 'brasilia-brasil':
        formatted_city = 'brasilia-df-brasil'
    if formatted_city == 'sao jose-brasil':
        formatted_city = 'sao jose-sc-brasil'

    return formatted_city

#criar um mapa básico centrado em uma localização do brasil
mapa = folium.Map(location=[-15.7801, -47.9292], zoom_start=4)

#adicionar marcadores ao mapa
for index, row in accommodation_counts.iterrows():
    #manipular o nome da cidade antes de fazer a pesquisa
    formatted_city_name = format_city_name(row['city_name'])
    
    #obter as coordenadas a partir do nome da cidade
    location = geolocator.geocode(formatted_city_name, timeout=10)

    #verificar se a localização foi encontrada
    if location:
        #mapear a nota para a cor do ícone
        color = map_nota_to_color(row['more_apartments_than_others'])

        #criar marcador com popup contendo informações e cor ajustada
        folium.Marker(
            location=[location.latitude, location.longitude],
            popup=f"{row['city_name']}",
            icon=folium.Icon(color=color, icon='home')
        ).add_to(mapa)

#exibir o mapa no notebook
display(mapa)

Análise 6 - Existem relações entre a nota do anúncio e os recursos disponíveis no imóvel?

In [ ]:
#Para esse vamos precisar fazer a média de todos os recursos de cada imóvel (já que cada quarto possui recursos diferentes) 
#e com a média de recursos de cada imóvel calcula a relação deles com a nota

def calcular_tamanho(lista_string):
    lista = ast.literal_eval(lista_string)
    return len(lista)

numeros_de_recursos = dados_details.loc[:,('hotel_id','room_facilities')]
numeros_de_recursos['num_of_facilities'] = numeros_de_recursos['room_facilities'].apply(calcular_tamanho)
numeros_de_recursos_media = numeros_de_recursos.loc[:,('hotel_id','num_of_facilities')].groupby('hotel_id').mean().reset_index()

In [ ]:
#Agora ligar o hotel_id e a nota (apenas hoteis que possuem alguma avaliação)

dados_hotel_nota= dados_ratings.loc[:,('hotel_id','Total')]
dados_hotel_nota.dropna(inplace=True)
dados_hotel_nota

In [ ]:
#Agora por fim,juntar os 2 dados e calcular sua correlação

dados_recursos_nota = pd.merge(dados_hotel_nota,numeros_de_recursos_media,on='hotel_id')
correlacao = dados_recursos_nota.loc[:,('Total', 'num_of_facilities')].corr()
print(correlacao) #0.146964 = fraca

In [ ]:
#vamos tentar observar isso de uma maneira mais visual, dividindo os hotel_id pela nota no caso(0-2,2-4,4-6,8-10) e usando a media de recursos como paramentro
#o que nos leva a uma observação mais clara que essa correlação é fraca de maneira mais visual

#definir as faixas de notas
faixas_notas = [0, 2, 4, 6, 8, 10]

#criar uma coluna 'faixa_notas' no DataFrame
dados_recursos_nota['faixa_notas'] = pd.cut(dados_recursos_nota['Total'], bins=faixas_notas, labels=[f'{i}-{i+2}' for i in range(0, 10, 2)])

#calcular a média de recursos para cada faixa de notas
media_recursos_por_faixa = dados_recursos_nota.groupby('faixa_notas')['num_of_facilities'].mean()

#plotar o gráfico de barras
media_recursos_por_faixa.plot(kind='bar', color='skyblue')
plt.title('Média de Recursos Normalizada por Faixa de Notas')
plt.xlabel('Faixa de Notas')
plt.ylabel('Média de Recursos (normalizada)')
plt.show()

In [ ]:
#E agora testando com recursos especificos('Ar-condicionado')
def tem_recurso(lista_string):
    lista = ast.literal_eval(lista_string)
    if 'Ar-condicionado'in lista:
        return True
    else:
        return False
    
dados_tem_ar = dados_details.loc[:,('hotel_id','room_facilities')]
dados_tem_ar['tem_ar'] = dados_tem_ar['room_facilities'].apply(tem_recurso)
dados_tem_ar_media = dados_tem_ar.loc[:,('hotel_id','tem_ar')]
dados_tem_ar_media.drop_duplicates(inplace=True)

dados_hotel_nota.dropna(inplace=True)
dados_ar_nota = pd.merge(dados_tem_ar_media,dados_hotel_nota,on='hotel_id')
dados_ar_nota.loc[:,('tem_ar','Total')].groupby('tem_ar').mean()

In [ ]:
#E agora testando com recursos especificos(Vista do mar)
def tem_recurso(lista_string):
    lista = ast.literal_eval(lista_string)
    if 'Vista do mar'in lista:
        return True
    else:
        return False
    
dados_tem_vista = dados_details.loc[:,('hotel_id','room_facilities')]
dados_tem_vista['tem_vista'] = dados_tem_vista['room_facilities'].apply(tem_recurso)
dados_tem_vista_media = dados_tem_vista.loc[:,('hotel_id','tem_vista')]
dados_tem_vista_media.drop_duplicates(inplace=True)

dados_hotel_nota.dropna(inplace=True)
dados_vista_nota = pd.merge(dados_tem_vista_media,dados_hotel_nota,on='hotel_id')
dados_vista_nota.loc[:,('tem_vista','Total')].groupby('tem_vista').mean()

Análise 7 - Existe alguma relação entre a nota recebida e a localização?

In [ ]:
#Para vizualizaçao disso vamos fazer a media de nota de cada cidade e depois disso plotar um mapa

dados_hotel_nota = dados_ratings.loc[:,('hotel_id','Total')]
dados_nota_cidade = pd.merge(dados_hotel_nota,dados_hotel_cidade,on='hotel_id')
dados_nota_media_por_cidade = dados_nota_cidade.loc[:,('city_name','Total')].groupby('city_name').mean().reset_index()
dados_nota_media_por_cidade.dropna(inplace=True)
dados_nota_media_por_cidade


In [ ]:
#configurar o geolocalizador sem verificação SSL
geolocator = Nominatim(user_agent="my_geocoder", ssl_context=None)

#função para mapear a nota para uma cor
def map_nota_to_color(nota):
    if 0 <= nota < 3:
        return 'red'
    elif 3 <= nota < 7:
        return 'orange'
    elif 7 <= nota < 9:
        return 'green'
    else:
        return 'purple'

#criar um mapa básico centrado em uma localização do brasil
mapa = folium.Map(location=[-15.7801, -47.9292], zoom_start=4)

#adicionar marcadores ao mapa
for index, row in dados_nota_media_por_cidade.iterrows():
    #manipular o nome da cidade antes de fazer a pesquisa
    formatted_city_name = format_city_name(row['city_name'])
    
    #obter as coordenadas a partir do nome da cidade
    location = geolocator.geocode(formatted_city_name, timeout=10)

    #verificar se a localização foi encontrada
    if location:
        # Mapear a nota para a cor do ícone
        color = map_nota_to_color(row['Total'])

        #criar marcador com popup contendo informações e cor ajustada
        folium.Marker(
            location=[location.latitude, location.longitude],
            popup=f"{row['city_name']}: {row['Total']}",
            icon=folium.Icon(color=color, icon='home')
        ).add_to(mapa)

#exibir o mapa no notebook
display(mapa)

Análise 8 - Conclusão sobre as notas dos imóveis

- De acordo com as analises anteriores podemos inferir:

- Imovéis localizados em cidades que possuem praia costumam fazer mais sucesso entre os demais.

- A quantidade de recursos disponiveis não necessariamente impacta sobre a nota,o que impacta de verdade é a qualidade e experiencia que o cliente vivenciouao utilizar algum desses recursos. 

- Imovéis localizados em cidades que possuem praia costumam fazer mais sucesso entre os demais. 

- A localização do imóvel importa muito na sua avaliação, já que também faz parte da experiencia do cliente.

Análise 9 - Anúncios críticos

- Todos os anúncios sem avaliação : anúncios sem avaliação deixam o cliente com um certo receio de ir para aquele local, já que ele não possui nenhuma referência daquele hotel.

- Anúncios com poucas avaliações e notas abaixo de 7: ainda com o mesmo motivo da passada, só que em uma situação mais complicada, porque além de possuir poucas opiniões, eles ainda são negativas,o que causa total receio por parte do cliente. 
